### taeyoonahn

**1. 연금보험료 측면 기존 유니콘 기업 데이터 분석**   
**&nbsp;&nbsp;1.1. 유니콘으로 인정받은 시점의 연금보험료 최소값     
&nbsp;&nbsp;1.2. 기업별 기간내 연금보험료 평균 상승율**

**2. 최종 산출물**   
**&nbsp;&nbsp;2.1. 유니콘으로 인정받은 시점의 연금보험료 최소값     
&nbsp;&nbsp;2.2. 기업별 기간내 연금보험료 평균 상승율**

In [45]:
#package 불러오기
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("font", family="Malgun Gothic")

from datetime import datetime
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings(action='ignore')

In [46]:
#실수는 소수점 두자리까지 표기
pd.options.display.float_format = '{:.2f}'.format

In [47]:
#데이터 불러오기
df_ori=pd.read_csv('unicorn.csv',engine = "python", encoding = "CP949")
df_ori

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
0,지피클럽,22474017,2015,11,733104.00,8
1,지피클럽,22474017,2015,12,745068.00,7
2,지피클럽,48329929,2016,1,551118.00,7
3,지피클럽,48329929,2016,2,488133.00,7
4,지피클럽,48329929,2016,3,1073632.00,7
...,...,...,...,...,...,...
545,옐로모바일,250700000,2019,8,1981617.00,11
546,옐로모바일,250700000,2019,9,1805970.00,11
547,옐로모바일,250700000,2019,10,1878560.00,11
548,옐로모바일,250700000,2019,11,2212953.00,12


In [48]:
#데이터 정보 확인
print(df_ori.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   회사ID      550 non-null    object 
 1   연매출액      550 non-null    int64  
 2   년도        550 non-null    int64  
 3   월         550 non-null    int64  
 4   월별_연금보험료  550 non-null    float64
 5   월별_직원수    550 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 25.9+ KB
None


In [49]:
#데이터 결측치 확인
df_ori.isnull()
df_ori.isnull().sum()

회사ID        0
연매출액        0
년도          0
월           0
월별_연금보험료    0
월별_직원수      0
dtype: int64

In [50]:
#연금보험료를 다루기 쉽게 데이터프레임 변경
df=df_ori[['회사ID','년도','월별_연금보험료']]
df

,회사ID,년도,월별_연금보험료
0,지피클럽,2015,733104.00
1,지피클럽,2015,745068.00
2,지피클럽,2016,551118.00
3,지피클럽,2016,488133.00
4,지피클럽,2016,1073632.00
...,...,...,...
545,옐로모바일,2019,1981617.00
546,옐로모바일,2019,1805970.00
547,옐로모바일,2019,1878560.00
548,옐로모바일,2019,2212953.00


In [51]:
#연금보험료를 다루기 쉽게 데이터프레임 변경 
#월별 연금보험료의 결측치를 고려해 연도별 월 평균 연금보험료로 바꿈
df_year = df.groupby(['회사ID','년도']).mean()
df_year.reset_index(inplace=True)
df_year.rename(columns = {'월별_연금보험료' : '연금보험료'}, inplace = True)
df_year

,회사ID,년도,연금보험료
0,무신사,2015,5921860.50
1,무신사,2016,7283040.33
2,무신사,2017,9954748.00
3,무신사,2018,16969701.42
4,무신사,2019,36105729.83
5,쏘카,2015,9692472.47
6,쏘카,2016,17036747.99
7,쏘카,2017,25767927.26
8,쏘카,2018,29599275.34
9,쏘카,2019,42750757.14


**1. 연금보험료 측면 기존 유니콘 기업 데이터 분석**

**1.1. 기업별 유니콘으로 인정받은 시점의 연금보험료 최소값**

> **기업별 유니콘으로 인정받은 시점**   
>
>**-무신사 2019.11   
-쏘카 2020.10   
-야놀자 2019.6   
-에이프로젠 2019.5   
-엘앤피코스메틱 2016.1   
-옐로모바일 2014.11   
-우아한형제들 2014.12   
-지피클럽 2018.10   
-쿠팡 2014.5   
-크래프톤 2018.8   
-토스 2018.12**

In [52]:
df_unicorn=pd.DataFrame(data={'회사ID':['무신사','쏘카','야놀자','에이프로젠','엘앤피코스메틱','옐로모바일','우아한형제들','지피클럽','쿠팡','크래프톤','토스'],'유니콘_등재년도':['2019','2020','2019','2019','2016','2014','2014','2018','2014','2018','2018']})

> **기업별 유니콘으로 인정받은 시점의 연금보험료**   
>
>**-무신사 36105729.83 
-쏘카 알수없음   
-야놀자 95933831.73   
-에이프로젠 18982074.43   
-엘앤피코스메틱 20327331.33   
-옐로모바일 5167708.50(2015)   
-우아한형제들 293398073.10(2015)   
-지피클럽 3796700.42   
-쿠팡 731149356.50(2015)   
-크래프톤 80043122.83   
-토스 247811184.05**

**1.2. 기업별 기간내 연금보험료 평균 상승율**

In [53]:
#기업별 연금보험료 변화율
df_year['연금보험료_작년대비_변화율']=df_year.sort_values(['년도']).groupby(['회사ID']).연금보험료.pct_change()
df_year.dropna(inplace=True)
df_year

,회사ID,년도,연금보험료,연금보험료_작년대비_변화율
1,무신사,2016,7283040.33,0.23
2,무신사,2017,9954748.00,0.37
3,무신사,2018,16969701.42,0.70
4,무신사,2019,36105729.83,1.13
6,쏘카,2016,17036747.99,0.76
7,쏘카,2017,25767927.26,0.51
8,쏘카,2018,29599275.34,0.15
9,쏘카,2019,42750757.14,0.44
11,야놀자,2016,22816351.51,0.46
12,야놀자,2017,32040208.50,0.40


In [54]:
#기업별 기간 내 연금보험료 누적변화율
df_year['연금보험료_누적_변화율']=df_year['연금보험료_작년대비_변화율']+1
df_year['연금보험료_누적_변화율']=df_year.sort_values(['년도']).groupby(['회사ID']).연금보험료_누적_변화율.cumprod()-1
df_year

,회사ID,년도,연금보험료,연금보험료_작년대비_변화율,연금보험료_누적_변화율
1,무신사,2016,7283040.33,0.23,0.23
2,무신사,2017,9954748.00,0.37,0.68
3,무신사,2018,16969701.42,0.70,1.87
4,무신사,2019,36105729.83,1.13,5.10
6,쏘카,2016,17036747.99,0.76,0.76
7,쏘카,2017,25767927.26,0.51,1.66
8,쏘카,2018,29599275.34,0.15,2.05
9,쏘카,2019,42750757.14,0.44,3.41
11,야놀자,2016,22816351.51,0.46,0.46
12,야놀자,2017,32040208.50,0.40,1.04


In [55]:
#기업별 기간 내 연금보험료 평균변화율
df_change=df_year.groupby(['회사ID']).연금보험료_누적_변화율.mean()
df_change=df_change.to_frame().reset_index()
df_change.rename(columns={'연금보험료_누적_변화율':'연금보험료_평균변화율'},inplace=True)
df_change.sort_values(by='연금보험료_평균변화율',ascending=False,inplace=True)
df_change.reset_index(inplace=True)
df_change.drop(['index'],axis=1,inplace=True)
df_change

,회사ID,연금보험료_평균변화율
0,토스,5.62
1,지피클럽,5.06
2,야놀자,2.16
3,우아한형제들,2.00
4,쏘카,1.97
5,무신사,1.97
6,엘앤피코스메틱,1.28
7,크래프톤,1.04
8,옐로모바일,0.92
9,에이프로젠,0.67


In [56]:
df_unicorn

,회사ID,유니콘_등재년도
0,무신사,2019
1,쏘카,2020
2,야놀자,2019
3,에이프로젠,2019
4,엘앤피코스메틱,2016
5,옐로모바일,2014
6,우아한형제들,2014
7,지피클럽,2018
8,쿠팡,2014
9,크래프톤,2018


In [57]:
df_unichange=pd.merge(df_change,df_unicorn)
df_unichange

,회사ID,연금보험료_평균변화율,유니콘_등재년도
0,토스,5.62,2018
1,지피클럽,5.06,2018
2,야놀자,2.16,2019
3,우아한형제들,2.00,2014
4,쏘카,1.97,2020
5,무신사,1.97,2019
6,엘앤피코스메틱,1.28,2016
7,크래프톤,1.04,2018
8,옐로모바일,0.92,2014
9,에이프로젠,0.67,2019


**2.최종산출물**

**2.1 유니콘으로 인정받은 시점의 연금보험료의 최소값**

> **지피클럽 3796700.42**

**2.2. 기업별 기간내 연금보험료 평균 상승율**

In [59]:
df_unichange

,회사ID,연금보험료_평균변화율,유니콘_등재년도
0,토스,5.62,2018
1,지피클럽,5.06,2018
2,야놀자,2.16,2019
3,우아한형제들,2.00,2014
4,쏘카,1.97,2020
5,무신사,1.97,2019
6,엘앤피코스메틱,1.28,2016
7,크래프톤,1.04,2018
8,옐로모바일,0.92,2014
9,에이프로젠,0.67,2019
